In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import rpad.visualize_3d.plots as pvp

In [ ]:
# pred_dir = "../results/ndf/mug/upright/release/2023-05-03_152512/predictions"
# pred_dir = "/home/beisner/code/rpad/taxpose/mug_place"
pred_dir = "/home/beisner/code/multi_project/taxpose/results/ndf/mug/upright/release/2023-05-03_220706/repro_mug"
result_ix = 0

In [ ]:
def load_and_print(fn):
    obj = np.load(fn)
    print(fn)
    for k, v in obj.items():
        print("\t", k, v.shape)
    return obj

In [ ]:
# These are all the files that get dumped.
init_obj_pts = load_and_print(f"{pred_dir}/{result_ix}_init_obj_points.npz")
# init_all_pts = load_and_print(f"{pred_dir}/{result_ix}_init_all_points.npz")
# post_grasp_all_pts = load_and_print(f"{pred_dir}/{result_ix}_post_grasp_all_points.npz")
post_grasp_obj_pts = load_and_print(f"{pred_dir}/{result_ix}_post_grasp_obj_points.npz")
# post_teleport_all_pts = load_and_print(f"{pred_dir}/{result_ix}_post_teleport_all_points.npz")
post_teleport_obj_pts = load_and_print(f"{pred_dir}/{result_ix}_post_teleport_obj_points.npz")
# pre_grasp_all_pts = load_and_print(f"{pred_dir}/{result_ix}_pre_grasp_all_points.npz")
pre_grasp_obj_pts = load_and_print(f"{pred_dir}/{result_ix}_pre_grasp_obj_points.npz")
# teleport_all_pts = load_and_print(f"{pred_dir}/{result_ix}_teleport_all_points.npz")
teleport_obj_pts = load_and_print(f"{pred_dir}/{result_ix}_teleport_obj_points.npz")

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=3,
    cols=2, 
    subplot_titles=[
        "init_obj_points",
        "pre_grasp_obj_points",
        "post_grasp_obj_points",
        "teleport_obj_points",
        "post_teleport_obj_points",
    ],
    specs=[
        [{"type": "scene"}, None],
        [{"type": "scene"}, {"type": "scene"}],
        [{"type": "scene"}, {"type": "scene"}],
    ],
    horizontal_spacing=0.05,
    vertical_spacing=0.05,
)

def add_seg_figure(data, labels, labelmap, scene, row, col):
    traces = pvp._segmentation_traces(
        data,
        labels,
        labelmap,
        scene,
        sizes=None
    )

    fig.add_traces(traces, rows=row, cols=col)
    fig.update_layout(**{
        scene:pvp._3d_scene(data),
        "showlegend":True,
        "margin":dict(l=0, r=0, b=0, t=40),
        "legend":dict(x=1.0, y=0.75),
    }
    )
    
def add_inter_seg_figure(npz_dict, scene, row, col, k=8000):
    ixs = np.random.permutation(len(npz_dict["clouds"]))[:k]
    add_seg_figure(
        npz_dict["clouds"][ixs],
        labels=npz_dict["classes"][ixs].astype(int),
        labelmap={0: "mug", 2: "gripper", 1: "rack"},
        scene=scene,
        row=row,
        col=col,
    )

# Input figure.
add_seg_figure(
    np.concatenate(
    [
        init_obj_pts["points_mug_raw"],
        init_obj_pts["points_gripper_raw"],
        init_obj_pts["points_rack_raw"],
    ],
    axis=1,
    ).squeeze(),
    labels=np.repeat([0, 1, 2], [1024, 1024, 1024]),
    labelmap={-1: "background", 0: "mug", 1: "gripper", 2: "rack"}, 
    scene="scene1",
    row=1,
    col=1,
)



add_inter_seg_figure(pre_grasp_obj_pts, "scene2", 2, 1)
add_inter_seg_figure(post_grasp_obj_pts, "scene3", 2, 2)
add_inter_seg_figure(teleport_obj_pts, "scene4", 3, 1)
add_inter_seg_figure(post_teleport_obj_pts, "scene5", 3, 2)

fig.update_layout(
    autosize=False,
    width=1600,
    height=2400,)


In [ ]:
fig.write_html(f"{pred_dir}/{result_ix}.html")